<a href="https://colab.research.google.com/github/ksonh/Vigenere-Cipher-BMP-Image-Writer/blob/main/Vigen%C3%A8re_Cipher_BMP_Image_Writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Writing Encrypted Text to BMP Image: A How-To, by Kenneth Hanson

#Step 1: Encrypt Text File with Vigenère Method

Before we write a single ASCII code to a byte, we need a way to encrypt our text file. For our purposes we will use the Vigenère method, named in honor of French cryptographer Blaise de la Vigenère. This cipher shifts the alphabet to the position of the character in the key. Take the key MATRIX, for instance. For the first character in the text, the alphabet starts at the position of letter M. For the second, it starts at letter A, and so on and so forth.  

Writing Functions to Encrypt and Decrypt

---



Our function will take two inputs: the text and the keyword. We will initiate an empty string called cipherText to which we will append the encrypted characters in the for loop. The keyword is capitalized for convenience. We initiate a counter called key_index to mark the index of the key character.

In [ ]:
def vigenereEncrypt(textInput, keyword):
   cipherText = ""
   keyword = keyword.upper()
   key_index = 0

Now we move on to the for loop which will iterate through each character of the text. To begin, we check if the character is a letter in the alphabet. If so, the shift is set to the index number of the input character from letter A. We do the same for the keyword character, and add it to the shift. For good measure we make sure the shift does not exceed 26, the length of the alphabet.

In [ ]:
for char in textInput:
   if char.isalpha():
       #Determines difference of input character from capital letter A in the ASCII table.
       shift = ord(char.upper()) - ord('A')
       #Adds by keyword character from capital letter A.
       shift += (ord(keyword[key_index]) - ord('A'))
       #Ensures shift does not exceed length of the alphabet.
       encrypt_shift = shift % 26


In the loop definition we add the new character with the shift. If it is an uppercase letter, we use the ASCII codes for capital letters. Otherwise, we use those for the lowercase ones. If the key_index equals the length of the keyword, we set it to 0.

In [ ]:
if char.isupper():
   cipherText += chr(encrypt_shift + ord('A'))
else:
   cipherText += chr(encrypt_shift + ord('a'))
key_index += 1
if key_index == len(keyword):
   key_index = 0

The full function definition would appear as follows:

In [ ]:
def vigenereEncrypt(textInput, keyword):
   cipherText = ""
   keyword = keyword.upper()
   key_index = 0

   for char in textInput:
      if char.isalpha():
         shift = ord(char.upper() - ord('A'))
         shift += (ord(keyword[key_index]) - ord('A'))
         encrypt_shift = shift % 26

         if char.isupper():
             cipherText += chr(encrypt_shift + ord('A'))
         else:
             cipherText += chr(encrypt_shift + ord('a'))
         key_index += 1
         if key_index == len(keyword):
            key_index = 0
      else:
         cipherText += char

   return cipherText

The function definition for decrypting runs in much the same way, except that we subtract by the index number of the keyword character from letter A, not add it.

In [ ]:
#Determines difference of input character from capital letter A in the ASCII table.
shift = ord(char.upper()) - ord('A')
#Subtracts by keyword character from capital letter A.
shift -= (ord(keyword[key_index]) - ord('A'))
#Ensures the shift does not exceed the length of the alphabet.
encrypt_shift = shift % 26

Combining Encryption and Decryption into One Function

---



For simplicity, can combine these tasks into one method, and change the behavior based on the mode. We'll do this with a named argument. The vigenereCipher() function will accept the text input, keyword, and mode as arguments. In the method definition, we initiate an empty string to which we will add the characters of the encrypted or decrypted text. The keyword will be set to its uppercase version, and the key_index will track the current position in the keyword.  A for loop will iterate through each character of the text input.

If the character is alphabetical, we find the difference of the ASCII code for that character in uppercase from that of capital A. In all other cases, we return the character, as with an empty space or a punctuation mark. When the mode is 'encrypt', we add the difference of the key_index character from capital A to the shift. Alternatively, when the mode is 'decrypt', we subtract that difference from the shift. Next, we ensure the shift does not exceed the length of the alphabet with a modulus. Finally, we determine whether the textInput character is uppercase or lowercase, and add the character of the ASCII code plus the shift to the result text. Then, we increment the key_index. If the key_index equals the length of the keyword, we set the key_index to 0. On completion we return the resulting text.

In [ ]:
def vigenereCipher(textInput, keyword, mode):
    resultText = ""
    keyword = keyword.upper()
    key_index = 0

    for char in textInput:
       if char.isalpha():
            shift = ord(char.upper()) - ord('A')

            if mode == 'encrypt':
               shift += (ord(keyword[key_index]) - ord('A'))
            elif mode == 'decrypt':
               shift -= (ord(keyword[key_index]) - ord('A'))

            encrypt_shift = shift % 26

            if char.isupper():
               resultText += chr(encrypt_shift + ord('A'))
            else:
               resultText += chr(encrypt_shift + ord('a'))

            key_index += 1
            if key_index == len(keyword):
              key_index = 0
       else:
           resultText += char

   return resultText

#Step 2: Break Down Process into Class

For modularity, we will define a class and call it ImageProcess. Our goal is to accept an encrypted text, parse through its characters, and write each character to a pixel according to its ordering. The program accepts a 24-bit BMP image and read the information from the header, including the size of the file in bytes, the start of the image data, the image width, and the image height. We need a function to compute the integer from four bytes using the equation $b_0+b_1\cdot 256+b_2⋅ 256^2 +b_3 ⋅ 256^3$. We assume the little-endian byte order is used. The readInt() function will read four bytes starting at the offset, then compute the integer. The code would be as follows:



In [ ]:
def read_int(offset):
    imageFile.seek(offset)

    theBytes = imageFile.read(4)
    result = 0
    base = 1
    for i in range(4):
       result = result + theBytes[i] * base
       base = base * 256

    return result

In our constructor we initiate the instance variables for the file size, the start of the image data, the image width, and the image height. This information will be found at the positions of 2, 10, 18, and 22 of the header, which we pass into the read_int() function call:

In [ ]:
fileSize = imageFile.read_int(2)
start = imageFile.read_int(10)
width = imageFile.read_int(18)
height = imageFile.read_int(22)

Using Python's random module, we generate a random number representing the index position we want to start writing the text. For our program we write a character at every prime index. The method to generate the first index will then be called generate_prime_start(). It accepts the lower and upper bounds of the range, which are be 10 and 99 in this case. To avoid the situation that this goes off the border, we pass in the minimum of 99 or the maxStartingPoint in the place of the upper bound. The maxStartingPoint will be the difference of the width and the offset:

In [ ]:
maxStartingPoint = width - start
startingPoint = generate_prime_start(10, min(99, maxStartingPoint))

At this point we almost have what we need to generate a user hint. This will be in the form of a string of numbers, including the starting point, skip order, and the text size. The text size is simply the length of the encrypted text. But for the skip order we need a way to communicate we are printing at every prime index. To make it straightforward, we set the skip order to the ASCII code representation for the word "PRIME", which is "8082737769". With all this combined, we print the hint to the console. We also compute the image size for good measure:

In [ ]:
textSize = len(encryptedChars)
skipOrder = "8082737769"
userHint = str(startingPoint) + skipOrder + str(textSize)
hintMessage = print(f'Here is your clue: {userHint}')
imageSize = width * height * 3

Next, we need to define a method to calculate the padding. Each row of the file is padded with additional bytes to make the number of bytes in the row divisible by four. The calculate_padding() function will accept two arguments: the current position of the file handle and the width of the row. We will assign this to a variable for us to use later:

In [ ]:
padding = calculate_padding(0, width)

Now there is not much else to do besides read the file. For this task, we have a nested loop. Using the if/else syntax, we either write the encrypted character or write the pixel as it read. This task can be made into a single function that changes its behavior based on whether the index of the column is prime or not. We create a function to determine that the number passed into it is prime, that is, that the column index is prime:

In [ ]:
for row in range(height):
   for col in range(width):
      if is_prime(col) == True:
         imageFile.write_pixel(prime=True)
      else:
         imageFile.write_pixel()

Finally, we move the file handle to skip over empty spaces and/or padding from its current position, then close the file:

In [ ]:
   padding = calculate_padding(imageFile.tell(), width)
   imageFile.seek(padding, SEEK_CUR)

imageFile.close()

# Step 3: Define Class and Methods

In the ImageProcess class there are four tasks we need to execute:  

1.   Calculate the padding from the current index to make number of bytes divisible by four.
2. Determine if a column index is a prime number.
3.   Generate a prime number index at which to start writing.
4. Write the pixel based on whether it is in a prime index position or not.
5. Read a sequence of four bytes to generate an integer representation.

All these tasks, of course, will be made into methods of the ImageProcess class. First, we need to define the instance variables in the constructor.





From step 2, we know there are four attributes we will need: the filename, width, start, and height. Since the cipher text is distinct to the image on which it is called, it should be made into an instance variable as well. We also need the image file of that image. An empty container for the encrypted characters is needed too. This is where we append the characters of the encrypted text in a for loop. In addition, we initialize an empty string for the Boolean value to check the message, which we will use later. Altogether, the class header and constructor would appear as so:

In [ ]:
class ImageProcess:
   def __init__(self, filename):
       self._filename = filename
       self._imageFile = ""
       self._cipherText = ""
       self._checkMessage = ""
       self._encryptedChars = []
       self._width = 0
       self._start = 0
       self._height = 0

Our class has no way of accessing the encrypted text we generated in the vigenereEncrypt() function. A method will not only have to pass in the cipher text, but loop through its characters and append each one to the self._encryptedChars container we initialized. We won't use the character itself, however, but its ASCII code:

In [ ]:
def enter_cipher_text(self, cipherText):
   self._cipherText = cipherText
   for i in range(len(cipherText)):
      self._encryptedChars.append(ord(cipherText[i]))

The main() method contains the logic of the image processing. First, it asks if the user wants the message to be printed to the console. Then, it extracts information from the image header, and prints the hint. A nested for loop iterates through each pixel and writes the byte triplet for blue, green, and red, changing its behavior based on whether the column index is prime or not. Whenever a character is written a counter is incremented. Finally, we check if they wanted the message to be printed, in which case we tell them to decrypt the message:

In [ ]:
def main(self):
   askCheck = input("Enter Y if you want to check the message, else enter N: ")
   self._checkMessage = askCheck

   #To open the binary file for reading and writing.
   self._imageFile = open(self._filename, "rb+")

   #To extract information from the image header.
   fileSize = self.read_int(2)
   self._start = self.read_int(10)
   self._width = self.read_int(18)
   self._height = self.read_int(22)

   maxStartingPoint = self._width - self._start
   startingPoint = self.generate_prime_start(10, min(99, maxStartingPoint))
   textSize = len(self._encryptedChars)
   skipOrder = "8082737769"
   userHint = str(startingPoint) + skipOrder + str(textSize)
   hintMessage = print(f'Here is your clue: {userHint}')
   imageSize = self._width * self._height * 3

   #To determine that scan lines occupy multiples of four bytes.
   padding - self.calculate_padding(0)

   charsCount = 0
   for row in range(self._height):
     for col in range(self._width):
        if self.is_prime(col):
           self.write_pixel(chars=self._encryptedChars, i=charsCount, checkMessage=self._checkMessage, prime=True)
           charsCount += 1
        else:
           self.write_pixel()

      #Moves file pointer to skip over empty spaces and/or padding.
      padding = self.calculate_padding(self._imageFile.tell(), self._width)
      self._imageFile.seek(padding, SEEK_CUR)

   self._imageFile.close()

   if self._checkMessage == "Y":
      print("Now decrypt the message according to the Vigenère cipher method.")


Next, we will define the calculate_padding() method, which takes the current_position and width of the row as arguments. At the start of the definition we calculate the line_size, which is the width of the row times three. Using the if/else syntax, we check if the current position plus the line size is divisible by 4, in which case we return a 0 for adjustment. If not, we determine how many bytes we need to reach the next multiple of 4:

In [ ]:
def calculate_padding(self, current_position):
   line_size = self._width * 3
   if (current_position + line_size) % 4 == 0:
      return 0
   else:
      return 4 - (current_position + line_size) % 4

Our next task is to determine if the index position of the column is prime, in which case we will return True, else we will return False. Using a for loop, we iterate from 2 to the square root of the number, given that for any composite number at least one factor must be less than or equal to the square root of the number. Inside the loop, we check if the number is divisible by the counter. If so, the number is not prime and the method returns False. In all other cases, the function returns True:

In [ ]:
def is_prime(self, col):
   if col < 2:
     return False
   for i in range(2, int(col**0.5) + 1):
      if col % i == 0:
         return False
   return True

According to our insertion scheme, we need to generate a prime number start. This function accepts the lower bound and upper bound as arguments. Using .randint() from the random module, we calculate a random number between those ranges, and continue doing so until the number is prime:

In [ ]:
def generate_prime_start(self, min_value, max_value):
   num = random.randint(min_value, max_value)

   while not self.isprime(num):
     num = random.randint(min_value, max_value)

   return num

Now we turn to the crucial task of writing the pixel. Recall from step two that we want to change the behavior of the writing based on whether the column index is prime or not. This can be achieved with a keyword argument set to a Boolean value. We also want a way to print out the encrypted message in the bytes objects. We'll do this with a keyword argument as well. A counter will be passed to indicate how many characters have been printed.

At the beginning of the method, we read three bytes, and assign each byte to its corresponding color. Afterwards, we check if the column is prime or the counter is less than the length of the chars list, in which case we move the file handle backwards by three bytes and write over the sequence of bytes it just read, writing the character in the position where the red byte was placed. If checkMessage is set to "Y", we print this byte object until we have iterated through the entire list of characters. In all other cases, we rewrite the same bytes object as read:

In [ ]:
def write_pixel(self, chars=[], i=0, checkMessage="N", prime=False):
    theBytes = self._imageFile.read(3)

    if len(theBytes) == 3:
       red = theBytes[0]
       green = theBytes[1]
       blue = theBytes[2]

       if prime == True and i < len(chars):
          self._imageFile.seek(-3, SEEK_CUR)
          self._imageFile.write(bytes([chars[i], green, blue]))
          if checkMessage == "Y":
             print(bytes([chars[i], green, blue]))
       else:
          self._imageFile.seek(-3, SEEK_CUR)
          self._imageFile.write(bytes([red, green, blue]))

Recall from step two that we have to read a sequences of four bytes from the header and convert them to little-endian integers. The read_int() method will accept the offset as an argument and move the file handle there. A sequence of four bytes ware read, then we iterate over the bytes, multiplying each byte with its power of 256, and adding it to the result. When finished, we return the result:

In [ ]:
def read_int(self, offset):
   self._imageFile.seek(offset)

   theBytes = self._imageFile.read(4)
   result = 0
   base = 1
   for i in range(4):
      result - result + theBytes[i] * base
      base = base * 256

   return result

# Step 4: Run and Test Program

Most of the program will have been complete after the class definition, except for creating a class object for the user. In the main, we ask for three inputs: the file name, user text, and user key. Then, we call the vigenereCipher() function and assign it to encryptedText, and call the same function for decryptedText but with the encryptedText. The program proceeds to construct the cipherImage object, and calls the .enter_cipher_text method with the encryptedText. Then, the body of the program is run with cipherImage.main().

The full imageprocessing.py program is included below.

In [ ]:
from io import SEEK_CUR
from sys import exit
from sys import argv
import random

class ImageProcess:
   """
   ImageProcess class for writing encrypted messages to
   pixels of image using the Vigenère cipher method.
   """
   def __init__(self, filename):
      """
      Initializes the ImageProcess object.

      Parameters:
      - filename(str): The name of the image file.
      """
      self._filename = filename
      self._imageFile = ""
      self._cipherText = ""
      self._checkMessage = ""
      self._encryptedChars = []
      self._width = 0
      self._start = 0
      self._height = 0

   def enter_cipher_text(self, cipherText):
      """
      Enters the cipher text for writing.

      Parameters:
      - cipherText (str): The encrypted message provided by the user.
      """
      self._cipherText = cipherText
      for i in range(len(cipherText)):
            # Takes encrypted text, converts each character to its ASCII code, and
            # appends the ASCII code to the empty list encryptedChars.
         self._encryptedChars.append(ord(cipherText[i]))

   def main(self):
      askCheck = input("Enter Y if you want to check the message, else enter N: ")
      self._checkMessage = askCheck

      # Opens the binary file for reading and writing.
      self._imageFile = open(self._filename, "rb+")

      # Extracts information from the image header.
      fileSize = self.read_int(2)
      self._start = self.read_int(10)
      self._width = self.read_int(18)
      self._height = self.read_int(22)

      maxStartingPoint = self._width - self._start
      startingPoint = self.generate_prime_start(10, min(99, maxStartingPoint))
      textSize = len(self._encryptedChars)
      skipOrder = "8082737769"
      userHint = str(startingPoint) + skipOrder + str(textSize)
      hintMessage = print(f'Here is your clue: {userHint}')
      imageSize = self._width * self._height * 3

      # Determines that scan lines occupy multiples of four bytes.
      padding = self.calculate_padding(0, self._width)

      charsCount = 0
      for row in range(self._height):
         for col in range(self._width):
            if self.is_prime(col):
               self.write_pixel(chars=self._encryptedChars,i=charsCount, checkMessage=self._checkMessage, prime=True)
               charsCount += 1
            else:
               self.write_pixel()

         # Moves file pointer to skip over empty spaces and/or padding.
         padding = self.calculate_padding(self._imageFile.tell(), self._width)
         self._imageFile.seek(padding, SEEK_CUR)

      self._imageFile.close()

      if self._checkMessage == "Y":
         print("Now decrypt the message according to the Vigenère cipher method.")

   def calculate_padding(self, current_position):
      """
      Calculates the padding necessary to align the current position to the next multiple of 4.

      Parameters:
      - current_position (int): The current index of the pointer.

      Returns:
      -int: Padding of 0 if the number of row pixels is a multiple of 4, else returns
            the number of bytes requires to align to the next multiple of 4.
      """
      # The total size of the row in bytes.
      line_size = self._width * 3
      # Checks if the total size of the row plus the
      # current position is already a multiple of 4.
      if (current_position + line_size) % 4 == 0:
         return 0
      else:
         # Determines how many bytes are required to reach the next
         # multiple of 4, subtracting that number from 4 to find the number
         # of bytes necessary to align to the next multiple of 4.
         return 4 - (current_position + line_size) % 4

   def is_prime(self, col):
      """
      Determines if the index position of the column is prime.

      Parameters:
      - col (int): The column position in the row.

      Returns:
      - bool: True if it is prime, else False.
      """
      if col < 2:
         return False
      for i in range(2, int(col**0.5) + 1):
         if col % i == 0:
            return False
      return True

   def generate_prime_start(self, min_value, max_value):
      """
      Generates a prime number index at which the first character of the cipher text is inserted.

      Parameters:
      - min_value (int): The lower bound of the range.
      - max_value (int): The upper bound of the range.

      Returns:
      - int: The random prime number integer.
      """
      num = random.randint(min_value, max_value)

      # Continues generating number until it is prime.
      while not self.is_prime(num):
         num = random.randint(min_value, max_value)

      return num

   def write_pixel(self, chars=[], i=0, checkMessage="N", prime=False):
      """
      Takes pixel and inserts character at the red byte representing
      8 bits if it is in a prime position in the row, until all characters
      in the encrypted message are stored. If not prime, it writes
      the original pixel.

      Parameters:
      - chars (list): The the list of characters or ASCII codes from the encrypted text.
      - i (int): The index in the chars list when called.
      - width (int): The width of the image row.
      - checkMessage (str): The keyword argument to indicate whether to print characters.
      - prime (bool): The keyword argument to indicate if pixel position is prime.
      """
      theBytes = self._imageFile.read(3)

      if len(theBytes) == 3:
         red = theBytes[0]
         green = theBytes[1]
         blue = theBytes[2]

         if prime == True and i < len(chars):
            self._imageFile.seek(-3, SEEK_CUR)
               # Inserts ASCII number of encrypted text character where the byte for red was placed.
            self._imageFile.write(bytes([chars[i], green, blue]))
            if checkMessage == "Y":
               print(bytes([chars[i], green, blue]))
         else:
            self._imageFile.seek(-3, SEEK_CUR)
            self._imageFile.write(bytes([red, green, blue]))

   def read_int(self, offset):
      """
      Converts a series of four bytes to a little-endian integer.

      Parameters:
      - offset (int): The offset to read the integer based on the header information.

      Returns:
      - int: The integer from the four bytes at that offset
             (2 for file size; 10 for offset; 18 for width; and 22 for height.)
      """
      self._imageFile.seek(offset)

      theBytes = self._imageFile.read(4)
      result = 0
      base = 1
      for i in range(4):
         result = result + theBytes[i] * base
         base = base * 256

      return result

def vigenereCipher(textInput, keyword, mode):
    """
    Converts the input text by the user and returns the encrypted or decrypted text according
    to the Vigenère cipher method with the shift of the uppercase character in the keyword.

    Parameters:
    - textInput (str): The character in the text to be encoded or decoded.
    - keyword (str): The keyword on which encryption or decryption shift is based.
    - mode (str): The mode indicating whether to 'encrypt' or 'decrypt'.

    Returns:
    - resultText (str): The resulting text of encryption or decryption.
    """
    resultText = ""
    keyword = keyword.upper()
    key_index = 0

    for char in textInput:
        if char.isalpha():
            # Determines difference of input character from capital letter A in the ASCII table.
            shift = ord(char.upper()) - ord('A')

            if mode == 'encrypt':
               # Adds by keyword character from capital letter A for encryption.
               shift += (ord(keyword[key_index]) - ord('A'))
            elif mode == 'decrypt':
               # Subtracts by keyword character from capital letter A for decryption.
               shift -= (ord(keyword[key_index]) - ord('A'))

            # Ensures the shift does not exceed the length of the alphabet of 26.
            encrypt_shift = shift % 26

            # Determines whether textInput character is uppercase or lowercase,
            # providing the offset to be used in calculation.
            if char.isupper():
               resultText += chr(encrypt_shift + ord('A'))
            else:
               resultText += chr(encrypt_shift + ord('a'))

            key_index += 1
            if key_index == len(keyword):
               key_index = 0
        else:
            resultText += char

    return resultText

if __name__ in '__main__':
   filename = input("Enter name of image file: ")
   userText = input("Enter the text you want to encrypt: ")
   userKey = input("Enter the key you want to use: ")

   encryptedText = vigenereCipher(userText, userKey)
   decryptedText = vigenereCipher(encryptedText, userKey)

   cipherImage = ImageProcess(filename)
   cipherImage.enter_cipher_text(encryptedText)
   cipherImage.main()